In [233]:
# Uses beautifulsoup4, urllib, json
# Before every commit, do "git pull --rebase"

'''first, get track ids from genre list

create dictionary with key( genre) and value( list of track ids )

for each track, get id, user_id, genre, tag_list
    for the user in the track:
        id, uri, permalink_url, country'''

from bs4 import BeautifulSoup
import urllib.request
import json
import csv as csv

page_url = "http://www.soundcloud.com/charts"
page_html = urllib.request.urlopen( page_url ).read()

page_beautifulsoup_object = BeautifulSoup( page_html, "html.parser" )

page_html_file = open( "html_saved_prettified.html", "w" )
page_html_file.write( page_beautifulsoup_object.body.div.find_all( "noscript" )[ 1 ].section.section.find_all( "article" )[ 1 ].ul.prettify() )

page_beautifulsoup_genres_array = page_beautifulsoup_object.body.div.find_all( "noscript" )[ 1 ].section.section.find_all( "article" )[ 1 ].ul

genre_song_list_length_limit = 100

api_url_chunk_1 = "https://api-v2.soundcloud.com/charts?kind=top&genre=soundcloud%3Agenres%3A"
api_url_chunk_2 = "&client_id=02gUJC0hH2ct1EGOcYXQIzRFU91c72Ea&limit=" + str( genre_song_list_length_limit ) + "&offset=0&linked_partitioning=1&app_version=1476434302"

genre_list = []

iterator = 0
for value in page_beautifulsoup_genres_array:
    if iterator > 3 and iterator % 2 == 1:
        # Reverse search through the HTML to find the first and last indices of the genre name
        genre_string_start_index = repr( value ).rfind( "=" ) + 1
        genre_string_end_index = repr( value ).rfind( "\"" )
        genre_string = repr( value )[ genre_string_start_index : genre_string_end_index ]
        genre_list.append( genre_string )
    iterator = iterator + 1

f = open("tracks1.csv","w")
f1 = csv.writer(f)
for genre in genre_list:
    json_data = urllib.request.urlopen( api_url_chunk_1 + genre + api_url_chunk_2 ).read().decode( "utf-8" )
    json_object = json.loads( json_data ).get( "collection" )
    
    for song_dict in json_object:
        #f1.writerow([song_dict["track"]["user_id"], song_dict["track"]["user"]["username"]])
        f1.writerow([song_dict["track"]["id"], song_dict["track"]["user_id"],song_dict["track"]["title"],genre,song_dict["track"]["playback_count"],song_dict["track"]["likes_count"]])

f.close()


In [235]:
import pandas as pd
df = pd.read_csv("tracks1.csv")
df.columns = ["track_id", "user_id", "title", "genre", "playback_count", "likes"]

In [167]:
df.drop_duplicates(inplace=True)

,track_id,user_id,title,genre,playback_count,likes
0,272105509,48084634,xxxtentacion - YuNg BrAtZ Prod. STAIN,alternativerock,1271007.0,28174.0
1,270817339,10494998,Stole Your Luv ++ Prod. Lyle LeDuff,alternativerock,4723663.0,94837.0
2,263994790,99685284,Hollywood Dreams / Comedown (Prod. by FKi & L...,alternativerock,3007916.0,50156.0
3,274770566,505203,Playboi Carti - Pump Fake! (Freestyle),alternativerock,1050026.0,28004.0
4,229528669,3932070,Emperor's New Clothes,alternativerock,5253830.0,79319.0


In [230]:
df.to_csv("tracks_and_artists.csv")

In [172]:
df.name.str.len().max()

50

In [174]:
df.shape

(2257, 2)

In [185]:
import string
printable  = set(string.printable)

In [272]:
a = lambda x: str(x).encode('ascii', 'replace')
b = lambda x: str.decode("utf-8").replace(u"\u2022", "*")

In [237]:
df["title"] = df["title"].apply(a)

In [244]:
df.dropna(inplace=True)

In [245]:
df.to_csv("tracks1.csv")

In [250]:
minuscomma = lambda x: str(x).replace(",", "-")

In [251]:
df["title"] = df["title"].apply(minuscomma)

In [253]:
df.to_csv("tracks1.csv")

In [266]:
tags =  pd.read_csv("tags.csv")

In [268]:
tags.columns = ["track_id", "tag"]
tags.head()

,track_id,tag
0,278372354,mc
1,278372354,pikachu
2,278372354,mc pedrinho
3,278372354,mc
4,278372354,pedrinho


In [273]:
tags["tag"] = tags["tag"].apply(a)

In [274]:
tags

,track_id,tag
0,278372354,b'mc'
1,278372354,b'pikachu'
2,278372354,b'mc pedrinho'
3,278372354,b'mc'
4,278372354,b'pedrinho'
5,278372354,b'mc'
6,278372354,b'mc'
7,278372354,b'dj r7'
8,278372354,b'dj'
9,278372354,b'r7'


In [275]:
tags.to_csv("tags.csv")

In [283]:
import pymssql

In [339]:
conn = pymssql.connect(server = 'dubhacks16.database.windows.net', database= 'SoundCloudInfo', user='dubs@dubhacks16', password= 'music16YEAH')

In [340]:
cursor = conn.cursor()

In [341]:
cursor.execute('SELECT distinct tag FROM tags')

In [342]:
row = cursor.fetchone()
tags = pd.DataFrame(columns=['tag'])
i = 0
while row:
    tags.loc[i] = minuscomma(row)
    row = cursor.fetchone()
    i+=1

In [344]:
tags.head()

,tag
0,('ercc'-)
1,('redit'-)
2,('mk'-)
3,('colonization of mars'-)
4,('kuldeep manak'-)


In [345]:
conn = pymssql.connect(server = 'dubhacks16.database.windows.net', database= 'SoundCloudInfo', user='dubs@dubhacks16', password= 'music16YEAH')
cursor = conn.cursor()
cursor.execute('SELECT distinct genre FROM tracks')
row = cursor.fetchone()
genres = pd.DataFrame(columns=['genre'])
i = 0
while row:
    genres.loc[i] = minuscomma(row)
    row = cursor.fetchone()
    i+=1
genres.head()

,genre
0,('trance'-)
1,('deephouse'-)
2,('latin'-)
3,('rock'-)
4,('comedy'-)


In [346]:
from sklearn import preprocessing
tag_id = preprocessing.LabelEncoder()
genre_id = preprocessing.LabelEncoder()

In [363]:
conn = pymssql.connect(server = 'dubhacks16.database.windows.net', database= 'SoundCloudInfo', user='dubs@dubhacks16', password= 'music16YEAH')
cursor = conn.cursor()
cursor.execute('SELECT tracks.id, tracks.title, tracks.user_id, tracks.genre, (SELECT followees.followee_id FROM (SELECT TOP (1) popular.followee_id, value FROM (SELECT COUNT(*) value, follows.followee_id FROM follows GROUP BY follows.followee_id ) popular ORDER BY popular.value DESC) followees) AS MostPopularfollowee_id, tracks.playback_count, tracks.like_count FROM Tracks tracks')
row = cursor.fetchone()
tracks = pd.DataFrame(columns=['id', 'title', 'user_id', 'genre', 'followee_id', 'playback_count'])
i = 0
while row:
    tracks.loc[i] = [row[0], row[1], row[2], row[3], row[4], row[5]]
    row = cursor.fetchone()
    i+=1
tracks.head()

,id,title,user_id,genre,followee_id,playback_count
0,311596.0,Gorillaz - feel good inc (drum and bass remix),58764.0,drumbass,222074365.0,646978.0
1,344109.0,Paint It Black,147446.0,rock,222074365.0,446519.0
2,554129.0,Pink Floyd - Shine On You Crazy Diamond (Parts...,164253.0,rock,222074365.0,1772765.0
3,567349.0,Linkin park - meteora - breaking the habit,239750.0,rock,222074365.0,3337786.0
4,609722.0,Mumford & Sons - Little Lion Man,37223.0,folksingersongwriter,222074365.0,4156253.0


In [364]:
tracks.shape

(4000, 6)

In [365]:
tracks.to_csv("data.csv")

In [358]:
tracks.columns

Index(['id', 'user_id', 'genre', 'tag', 'followee_id', 'playback_count',
       'likes'],
      dtype='object')